# Script Contents
#### .01 Import Libraries & Data
#### .02 Clean Data for Deep Learning
#### .03 Reshape Data for Deep Learning
#### .04 Split Data into Training and Testing Sets
#### .05 Create Keras Model
#### .06 Compile & Run RNN Model
#### .07 Confusion Matrix of Model Results
#### .08 Recreating Steps 5-7 with Adjusted Hyperparameters
#### .09 Adjusting Hyperparameters & Changing Activation
#### .10 Adjusting Hyperparameters, Adding Convolution and Pooling Layers, and Changing Optimizer
#### .11 Returning to Original Keras Setup with Adjusted Hyperparameters2 
#### 213 CNN Model
#### 314 CNN Model Retest
#### 415 CNN Final Test

## .01 Import Libraries & Data

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import operator
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from numpy import unique
from numpy import reshape
from keras.models import Sequential
from keras.layers import Conv1D, Conv2D, Dense, BatchNormalization, Flatten, MaxPooling1D, Dropout, LSTM
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [4]:
#Create a path to where your data is stored.
path = r'C:\Users\jacks\ClimateWins\02 Data Sets'

In [5]:
# Import unscaled weather data
df_unscaled = pd.read_csv(os.path.join(path, 'Dataset-weather-prediction-dataset-processed.csv'))

In [6]:
# Import pleasant weather data
df_pleasant = pd.read_csv(os.path.join(path, 'Dataset-Answers-Weather_Prediction_Pleasant_Weather.csv'))

## .02 Clean Data for Deep Learning

In [8]:
df_unscaled.head()

,DATE,MONTH,BASEL_cloud_cover,BASEL_wind_speed,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_snow_depth,BASEL_sunshine,...,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_snow_depth,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,19600101,1,7,2.1,0.85,1.018,0.32,0.09,0,0.7,...,5,0.88,1.0003,0.45,0.34,0,4.7,8.5,6.0,10.9
1,19600102,1,6,2.1,0.84,1.018,0.36,1.05,0,1.1,...,7,0.91,1.0007,0.25,0.84,0,0.7,8.9,5.6,12.1
2,19600103,1,8,2.1,0.90,1.018,0.18,0.30,0,0.0,...,7,0.91,1.0096,0.17,0.08,0,0.1,10.5,8.1,12.9
3,19600104,1,3,2.1,0.92,1.018,0.58,0.00,0,4.1,...,7,0.86,1.0184,0.13,0.98,0,0.0,7.4,7.3,10.6
4,19600105,1,6,2.1,0.95,1.018,0.65,0.14,0,5.4,...,3,0.80,1.0328,0.46,0.00,0,5.7,5.7,3.0,8.4


In [9]:
df_pleasant.head()

,DATE,BASEL_pleasant_weather,BELGRADE_pleasant_weather,BUDAPEST_pleasant_weather,DEBILT_pleasant_weather,DUSSELDORF_pleasant_weather,HEATHROW_pleasant_weather,KASSEL_pleasant_weather,LJUBLJANA_pleasant_weather,MAASTRICHT_pleasant_weather,MADRID_pleasant_weather,MUNCHENB_pleasant_weather,OSLO_pleasant_weather,SONNBLICK_pleasant_weather,STOCKHOLM_pleasant_weather,VALENTIA_pleasant_weather
0,19600101,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,19600102,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,19600103,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,19600104,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,19600105,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [10]:
# Drop date and month columns from df_unscaled
df_unscaled = df_unscaled.drop(columns=['DATE', 'MONTH'], axis=1)
df_unscaled.head()

,BASEL_cloud_cover,BASEL_wind_speed,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_snow_depth,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,...,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_snow_depth,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,7,2.1,0.85,1.018,0.32,0.09,0,0.7,6.5,0.8,...,5,0.88,1.0003,0.45,0.34,0,4.7,8.5,6.0,10.9
1,6,2.1,0.84,1.018,0.36,1.05,0,1.1,6.1,3.3,...,7,0.91,1.0007,0.25,0.84,0,0.7,8.9,5.6,12.1
2,8,2.1,0.90,1.018,0.18,0.30,0,0.0,8.5,5.1,...,7,0.91,1.0096,0.17,0.08,0,0.1,10.5,8.1,12.9
3,3,2.1,0.92,1.018,0.58,0.00,0,4.1,6.3,3.8,...,7,0.86,1.0184,0.13,0.98,0,0.0,7.4,7.3,10.6
4,6,2.1,0.95,1.018,0.65,0.14,0,5.4,3.0,-0.7,...,3,0.80,1.0328,0.46,0.00,0,5.7,5.7,3.0,8.4


In [11]:
# Drop date column from df_pleasant
df_pleasant = df_pleasant.drop(columns=['DATE'], axis=1)
df_pleasant.head()

,BASEL_pleasant_weather,BELGRADE_pleasant_weather,BUDAPEST_pleasant_weather,DEBILT_pleasant_weather,DUSSELDORF_pleasant_weather,HEATHROW_pleasant_weather,KASSEL_pleasant_weather,LJUBLJANA_pleasant_weather,MAASTRICHT_pleasant_weather,MADRID_pleasant_weather,MUNCHENB_pleasant_weather,OSLO_pleasant_weather,SONNBLICK_pleasant_weather,STOCKHOLM_pleasant_weather,VALENTIA_pleasant_weather
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [12]:
# Drop all Gdansk, Roma,Tours columns from df_unscaled since they are not included in pleasant weather data
# We know this from previous exercises using the same data
cols_to_drop = [col for col in df_unscaled.columns if col.startswith(('GDANSK', 'ROMA', 'TOURS'))]
df_unscaled = df_unscaled.drop(columns=cols_to_drop)

In [13]:
df_unscaled.columns

Index(['BASEL_cloud_cover', 'BASEL_wind_speed', 'BASEL_humidity',
       'BASEL_pressure', 'BASEL_global_radiation', 'BASEL_precipitation',
       'BASEL_snow_depth', 'BASEL_sunshine', 'BASEL_temp_mean',
       'BASEL_temp_min',
       ...
       'VALENTIA_cloud_cover', 'VALENTIA_humidity', 'VALENTIA_pressure',
       'VALENTIA_global_radiation', 'VALENTIA_precipitation',
       'VALENTIA_snow_depth', 'VALENTIA_sunshine', 'VALENTIA_temp_mean',
       'VALENTIA_temp_min', 'VALENTIA_temp_max'],
      dtype='object', length=147)

In [14]:
# Trying to find out all the different measurement types for each location

# Extract location names 
locations = set([col.split('_')[0] for col in df_unscaled.columns])

# Create a dictionary to store measurement counts for each location
measurement_counts = {location: {} for location in locations}

# Count occurrences of each measurement type for each location
for col in df_unscaled.columns:
    parts = col.split('_') 
    location = parts[0] 
    measurement = '_'.join(parts[1:])  # Join remaining parts if there are more than two

    if measurement not in measurement_counts[location]:
        measurement_counts[location][measurement] = 1
    else:
        measurement_counts[location][measurement] += 1

# Print the measurement counts for each location
for location, measurements in measurement_counts.items():
    print(f"Location: {location}")
    for measurement, count in measurements.items():
        print(f"  - {measurement}: {count}")
    print()

Location: MUNCHENB
  - cloud_cover: 1
  - humidity: 1
  - global_radiation: 1
  - precipitation: 1
  - snow_depth: 1
  - sunshine: 1
  - temp_mean: 1
  - temp_min: 1
  - temp_max: 1

Location: BASEL
  - cloud_cover: 1
  - wind_speed: 1
  - humidity: 1
  - pressure: 1
  - global_radiation: 1
  - precipitation: 1
  - snow_depth: 1
  - sunshine: 1
  - temp_mean: 1
  - temp_min: 1
  - temp_max: 1

Location: DEBILT
  - cloud_cover: 1
  - wind_speed: 1
  - humidity: 1
  - pressure: 1
  - global_radiation: 1
  - precipitation: 1
  - sunshine: 1
  - temp_mean: 1
  - temp_min: 1
  - temp_max: 1

Location: LJUBLJANA
  - cloud_cover: 1
  - wind_speed: 1
  - humidity: 1
  - pressure: 1
  - global_radiation: 1
  - precipitation: 1
  - sunshine: 1
  - temp_mean: 1
  - temp_min: 1
  - temp_max: 1

Location: MAASTRICHT
  - cloud_cover: 1
  - wind_speed: 1
  - humidity: 1
  - pressure: 1
  - global_radiation: 1
  - precipitation: 1
  - sunshine: 1
  - temp_mean: 1
  - temp_min: 1
  - temp_max: 1

Locat

Most measurements at a single location is 11 and they are:
  - cloud_cover: 1
  - wind_speed: 1
  - humidity: 1
  - pressure: 1
  - global_radiation: 1
  - precipitation: 1
  - snow_depth: 1
  - sunshine: 1
  - temp_mean: 1
  - temp_min: 1
  - temp_max: 1

- Missing
- Maastricht: snow_depth
- Madrid: snow_depth
- Heathrow: wind_speed measurements
- Kassel: cloud_cover, snow_depth
- Belgrade: wind_speed, snow_depth
- Valentia: wind_speed
- Budapest: wind_speed, snow_depth
- Ljubljana: snow_depth
- Sonnblick: snow_depth
- Stockholm: wind_speed, humidity, snow_depth
- Debilt: snow_depth
- Munchenb wind_speed, pressure
s missing 

In [16]:
# Drop columns for wind_speed and snow_depth measurements since they are missing so often

# Create a list of columns to drop
cols_to_drop = [col for col in df_unscaled.columns if col.endswith(('wind_speed', 'snow_depth'))]

# Dropping
df_unscaled = df_unscaled.drop(columns=cols_to_drop)

In [17]:
df_unscaled.head()

,BASEL_cloud_cover,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,BASEL_temp_max,BELGRADE_cloud_cover,...,STOCKHOLM_temp_max,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,7,0.85,1.018,0.32,0.09,0.7,6.5,0.8,10.9,1,...,4.9,5,0.88,1.0003,0.45,0.34,4.7,8.5,6.0,10.9
1,6,0.84,1.018,0.36,1.05,1.1,6.1,3.3,10.1,6,...,5.0,7,0.91,1.0007,0.25,0.84,0.7,8.9,5.6,12.1
2,8,0.90,1.018,0.18,0.30,0.0,8.5,5.1,9.9,6,...,4.1,7,0.91,1.0096,0.17,0.08,0.1,10.5,8.1,12.9
3,3,0.92,1.018,0.58,0.00,4.1,6.3,3.8,10.6,8,...,2.3,7,0.86,1.0184,0.13,0.98,0.0,7.4,7.3,10.6
4,6,0.95,1.018,0.65,0.14,5.4,3.0,-0.7,6.0,8,...,4.3,3,0.80,1.0328,0.46,0.00,5.7,5.7,3.0,8.4


In [18]:
# There are missing measurements for Kassel's cloud cover, Stockholm's humidity, and Munchenb's pressure
# We know that Ljubljana is near Kassel, Sonnblick is near Munchenb, and Olso is close enough to Stockholm

# Define relationships between locations
location_pairs = {
    'KASSEL': 'LJUBLJANA',
    'STOCKHOLM': 'OSLO',
    'MUNCHENB': 'SONNBLICK'
}

# Define the desired order of measurements
measurement_order = ['cloud_cover', 'humidity', 'pressure', 'global_radiation', 
                     'precipitation', 'sunshine', 'temp_mean', 'temp_min', 'temp_max']

# Function to fill missing values and insert in correct position
def fill_missing_values(df_unscaled, location, measurement, neighbor):
    """
    Fills missing values for a given location and measurement using data from a neighbor location.
    Inserts the new column in the correct position based on the measurement order.

    Args:
        df_unscaled: The DataFrame containing the weather data.
        location: The location with missing values.
        measurement: The measurement with missing values.
        neighbor: The neighboring location to use for filling.

    Returns:
        The updated DataFrame with filled missing values and columns in the correct order.
    """
    source_col = f'{neighbor}_{measurement}'
    target_col = f'{location}_{measurement}'

    # Determine the insertion index 
    if measurement == measurement_order[0]:  # If it's the first measurement for the location
        # Find the index of the first column for the location (or 0 if no location columns exist)
        location_columns = [col for col in df_unscaled.columns if col.startswith(location)]
        if location_columns:
            insert_index = df_unscaled.columns.get_loc(location_columns[0]) 
        else:
            insert_index = 0
    else:
        insert_index = df_unscaled.columns.get_loc(f'{location}_{measurement_order[measurement_order.index(measurement) - 1]}') + 1 

    # Create the new column with missing values and insert it at the correct position
    df_unscaled.insert(insert_index, target_col, np.nan) 

    # Fill missing values in the new column
    df_unscaled[target_col].fillna(df_unscaled[source_col], inplace=True) 

    return df_unscaled

# Fill missing values for each location and measurement
for location, neighbor in location_pairs.items():
    for measurement in measurement_order:
        if f'{location}_{measurement}' not in df_unscaled.columns:  # Check if column already exists
            df_unscaled = fill_missing_values(df_unscaled, location, measurement, neighbor)

# Checking new columns for existance and location
selected_columns = [col for col in df_unscaled.columns if col.startswith(('KASSEL', 'STOCKHOLM', 'MUNCHENB'))]
print(df_unscaled[selected_columns])

       KASSEL_cloud_cover  KASSEL_humidity  KASSEL_pressure  \
0                     8.0             0.82           1.0094   
1                     6.0             0.86           1.0086   
2                     8.0             0.91           1.0129   
3                     6.0             0.87           1.0290   
4                     7.0             0.86           1.0262   
...                   ...              ...              ...   
22945                 4.0             0.77           1.0161   
22946                 3.0             0.77           1.0161   
22947                 3.0             0.77           1.0161   
22948                 3.0             0.77           1.0161   
22949                 3.0             0.77           1.0161   

       KASSEL_global_radiation  KASSEL_precipitation  KASSEL_sunshine  \
0                         0.28                  0.48              1.6   
1                         0.12                  0.27              0.0   
2                       

C:\Users\jacks\AppData\Local\Temp\ipykernel_9624\1246060646.py:48: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_unscaled[target_col].fillna(df_unscaled[source_col], inplace=True)
C:\Users\jacks\AppData\Local\Temp\ipykernel_9624\1246060646.py:48: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behave

In [19]:
# Checking new columns for existance and location
selected_columns = [col for col in df_unscaled.columns if col.startswith(('MUNCHENB'))]
print(df_unscaled[selected_columns])

       MUNCHENB_cloud_cover  MUNCHENB_humidity  MUNCHENB_pressure  \
0                         5               0.67             1.0304   
1                         6               0.72             1.0292   
2                         6               0.91             1.0320   
3                         6               0.90             1.0443   
4                         5               0.85             1.0430   
...                     ...                ...                ...   
22945                     2               0.76             1.0263   
22946                     6               0.70             1.0263   
22947                     7               0.64             1.0263   
22948                     6               0.75             1.0263   
22949                     5               0.83             1.0263   

       MUNCHENB_global_radiation  MUNCHENB_precipitation  MUNCHENB_sunshine  \
0                           0.20                    0.10                0.0   
1            

In [20]:
df_unscaled.shape

(22950, 135)

In [21]:
df_pleasant.shape

(22950, 15)

In [22]:
# Export cleaned weather data
df_unscaled.to_csv(os.path.join(path, 'weather_clean.csv'), index=False)

## .03 Reshape Data for Deep Learning

In [24]:
# Rename df's
X = df_unscaled
y = df_pleasant

In [25]:
# Convert df's to arrays
X = np.array(X)
y = np.array(y)

In [26]:
X

array([[ 7.    ,  0.85  ,  1.018 , ...,  8.5   ,  6.    , 10.9   ],
       [ 6.    ,  0.84  ,  1.018 , ...,  8.9   ,  5.6   , 12.1   ],
       [ 8.    ,  0.9   ,  1.018 , ..., 10.5   ,  8.1   , 12.9   ],
       ...,
       [ 4.    ,  0.76  ,  1.0227, ..., 10.7   ,  7.9   , 13.5   ],
       [ 5.    ,  0.8   ,  1.0212, ..., 10.7   ,  7.9   , 13.5   ],
       [ 5.    ,  0.84  ,  1.0193, ..., 10.7   ,  7.9   , 13.5   ]])

In [27]:
y

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [28]:
# Reshaping X as a 3D object
X = X.reshape(-1,15,9)

In [29]:
X.shape

(22950, 15, 9)

In [30]:
y.shape

(22950, 15)

## .04 Split Data into Training and Testing Sets

In [32]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 42)

## .05 Create Keras Model

In [34]:
epochs = 20
batch_size = 16
n_hidden = 16

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='sigmoid'))

C:\Users\jacks\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [35]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 16)             │         1,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 15)             │           255 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,919 (7.50 KB)

 Trainable params: 1,919 (7.50 KB)

 Non-trainable params: 0 (0.00 B)

## .06 Compile & Run RNN Model

In [37]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [38]:
model.fit(X_train,
          y_train,
          batch_size=batch_size,
          validation_data=(X_test, y_test),
          epochs=epochs)

Epoch 1/20
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.0895 - loss: 9.3875 - val_accuracy: 0.0516 - val_loss: 8.4701
Epoch 2/20
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.0765 - loss: 9.5785 - val_accuracy: 0.1656 - val_loss: 8.7969
Epoch 3/20
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.0854 - loss: 9.9517 - val_accuracy: 0.0418 - val_loss: 9.1275
Epoch 4/20
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.0909 - loss: 10.2187 - val_accuracy: 0.0465 - val_loss: 9.3693
Epoch 5/20
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.0883 - loss: 10.4448 - val_accuracy: 0.0462 - val_loss: 9.6446
Epoch 6/20
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.0838 - loss: 10.6621 - val_accuracy: 0.0802 - val_loss: 9.9237
Epoch 7/20
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.0897 - loss: 10.6931 - val_accuracy: 0.0380 - val_loss: 10.2171
Epoch 8/20
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.0849 - loss: 11.0

## .07 Confusion Matrix of Model Results

In [40]:
# Define list of stations names
stations = {
0: 'BASEL',
1: 'BELGRADE',
2: 'BUDAPEST',
3: 'DEBILT',
4: 'DUSSELDORF',
5: 'HEATHROW',
6: 'KASSEL',
7: 'LJUBLJANA',
8: 'MAASTRICHT',
9: 'MADRID',
10: 'MUNCHENB',
11: 'OSLO',
12: 'SONNBLICK',
13: 'STOCKHOLM',
14: 'VALENTIA'
}

In [41]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [42]:
# Evaluate
print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Pred        BASEL  BUDAPEST  DUSSELDORF  MADRID  MUNCHENB
True                                                     
BASEL        3680         1           1       0         0
BELGRADE     1083         2           2       5         0
BUDAPEST      212         1           0       1         0
DEBILT         82         0           0       0         0
DUSSELDORF     28         0           1       0         0
HEATHROW       81         1           0       0         0
KASSEL         11         0           0       0         0
LJUBLJANA      58         2           0       0         1
MAASTRICHT      9         0           0       0         0
MADRID        444         2           8       2         2
MUNCHENB        8         0           0       0         0
OSLO            4         0           0       1         0
STOCKHOLM       4         0           0       0         0
VALENTIA        1         0           0       0         0


## .08 Recreating Steps 5-7 with Adjusted Hyperparameters

In [44]:
epochs = 30
batch_size = 16
n_hidden = 32

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='sigmoid'))

C:\Users\jacks\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [45]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                   │ (None, 32)             │         5,376 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 15)             │           495 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,871 (22.93 KB)

 Trainable params: 5,871 (22.93 KB)

 Non-trainable params: 0 (0.00 B)

In [46]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [47]:
model.fit(X_train,
          y_train,
          batch_size=batch_size,
          validation_data=(X_test, y_test),
          epochs=epochs)

Epoch 1/30
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.1114 - loss: 10.4722 - val_accuracy: 0.0444 - val_loss: 8.7799
Epoch 2/30
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.1199 - loss: 10.5821 - val_accuracy: 0.0458 - val_loss: 9.3464
Epoch 3/30
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.1054 - loss: 10.9103 - val_accuracy: 0.0518 - val_loss: 9.8370
Epoch 4/30
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.0948 - loss: 11.1960 - val_accuracy: 0.0803 - val_loss: 10.4056
Epoch 5/30
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.0939 - loss: 11.5418 - val_accuracy: 0.0446 - val_loss: 11.0271
Epoch 6/30
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.0848 - loss: 12.3307 - val_accuracy: 0.0798 - val_loss: 11.4447
Epoch 7/30
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.0823 - loss: 12.3309 - val_accuracy: 0.0423 - val_loss: 10.5470
Epoch 8/30
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.0744 - loss

In [48]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [49]:
# Evaluate
print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Pred        BASEL  BUDAPEST  LJUBLJANA
True                                  
BASEL        3681         1          0
BELGRADE     1092         0          0
BUDAPEST      214         0          0
DEBILT         82         0          0
DUSSELDORF     29         0          0
HEATHROW       81         0          1
KASSEL         11         0          0
LJUBLJANA      61         0          0
MAASTRICHT      9         0          0
MADRID        456         1          1
MUNCHENB        8         0          0
OSLO            5         0          0
STOCKHOLM       4         0          0
VALENTIA        1         0          0


#### Accuracy appears to decrease while loss increase. 

## .09 Adjusting Hyperparameters & Changing Activation

In [52]:
epochs = 30
batch_size = 16
n_hidden = 64

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='tanh'))

C:\Users\jacks\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [53]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 64)             │        18,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 15)             │           975 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,919 (77.81 KB)

 Trainable params: 19,919 (77.81 KB)

 Non-trainable params: 0 (0.00 B)

In [54]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [55]:
model.fit(X_train,
          y_train,
          batch_size=batch_size,
          validation_data=(X_test, y_test),
          epochs=epochs)

Epoch 1/30
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.1225 - loss: 24.7744 - val_accuracy: 0.3989 - val_loss: 25.9336
Epoch 2/30
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.2126 - loss: 25.4424 - val_accuracy: 0.3980 - val_loss: 30.0702
Epoch 3/30
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.1140 - loss: 24.9259 - val_accuracy: 0.0051 - val_loss: 23.5693
Epoch 4/30
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.0471 - loss: 25.0565 - val_accuracy: 0.0258 - val_loss: 18.8678
Epoch 5/30
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.0492 - loss: 24.7410 - val_accuracy: 0.0734 - val_loss: 23.6034
Epoch 6/30
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.0487 - loss: 24.9295 - val_accuracy: 0.0484 - val_loss: 23.1586
Epoch 7/30
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.0774 - loss: 24.6685 - val_accuracy: 0.1391 - val_loss: 26.5976
Epoch 8/30
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.0471 - l

In [56]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [57]:
# Evaluate
print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Pred        BASEL  BUDAPEST  DEBILT
True                               
BASEL           1       114    3567
BELGRADE        0         0    1092
BUDAPEST        0         0     214
DEBILT          0         0      82
DUSSELDORF      0         0      29
HEATHROW        0         0      82
KASSEL          0         0      11
LJUBLJANA       0         0      61
MAASTRICHT      0         0       9
MADRID          0         0     458
MUNCHENB        0         0       8
OSLO            0         0       5
STOCKHOLM       0         0       4
VALENTIA        0         0       1


## .10 Adjusting Hyperparameters, Adding Convolution and Pooling Layers, and Changing Optimizer

In [59]:
epochs = 25
batch_size = 16
n_hidden = 8

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(MaxPooling1D())
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='tanh'))

C:\Users\jacks\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\jacks\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [60]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 14, 8)          │           152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 7, 8)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 8)              │           544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 8)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 15)             │           135 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 831 (3.25 KB)

 Trainable params: 831 (3.25 KB)

 Non-trainable params: 0 (0.00 B)

In [61]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [62]:
model.fit(X_train,
          y_train,
          batch_size=batch_size,
          validation_data=(X_test, y_test),
          epochs=epochs)

Epoch 1/25
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.6428 - loss: nan - val_accuracy: 0.6417 - val_loss: nan
Epoch 2/25
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.6414 - loss: nan - val_accuracy: 0.6417 - val_loss: nan
Epoch 3/25
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.6476 - loss: nan - val_accuracy: 0.6417 - val_loss: nan
Epoch 4/25
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.6417 - loss: nan - val_accuracy: 0.6417 - val_loss: nan
Epoch 5/25
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.6399 - loss: nan - val_accuracy: 0.6417 - val_loss: nan
Epoch 6/25
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.6470 - loss: nan - val_accuracy: 0.6417 - val_loss: nan
Epoch 7/25
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.6396 - loss: nan - val_accuracy: 0.6417 - val_loss: nan
Epoch 8/25
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.6440 - loss: nan - val_accuracy: 0.6417 - val_loss: nan
Epoch 9/

In [63]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [64]:
# Evaluate
print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Pred        BASEL
True             
BASEL        3682
BELGRADE     1092
BUDAPEST      214
DEBILT         82
DUSSELDORF     29
HEATHROW       82
KASSEL         11
LJUBLJANA      61
MAASTRICHT      9
MADRID        458
MUNCHENB        8
OSLO            5
STOCKHOLM       4
VALENTIA        1


## .11 Returning to Original Keras Setup with Adjusted Hyperparameters

In [190]:
epochs = 10
batch_size = 4
n_hidden = 4

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='sigmoid'))

C:\Users\jacks\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [192]:
model.summary()

Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_10 (LSTM)                  │ (None, 4)              │           224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 4)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 15)             │            75 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 299 (1.17 KB)

 Trainable params: 299 (1.17 KB)

 Non-trainable params: 0 (0.00 B)

In [194]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [196]:
model.fit(X_train,
          y_train,
          batch_size=batch_size,
          validation_data=(X_test, y_test),
          epochs=epochs)

Epoch 1/10
4303/4303 ━━━━━━━━━━━━━━━━━━━━ 13s 3ms/step - accuracy: 0.1078 - loss: 8.8330 - val_accuracy: 0.0786 - val_loss: 8.7949
Epoch 2/10
4303/4303 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - accuracy: 0.0730 - loss: 9.4860 - val_accuracy: 0.0791 - val_loss: 9.5490
Epoch 3/10
4303/4303 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - accuracy: 0.0525 - loss: 10.2104 - val_accuracy: 0.0798 - val_loss: 10.1730
Epoch 4/10
4303/4303 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - accuracy: 0.0466 - loss: 11.1597 - val_accuracy: 0.0800 - val_loss: 10.9902
Epoch 5/10
4303/4303 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - accuracy: 0.0487 - loss: 11.8088 - val_accuracy: 0.0796 - val_loss: 11.4661
Epoch 6/10
4303/4303 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - accuracy: 0.0459 - loss: 12.7693 - val_accuracy: 0.0791 - val_loss: 11.9658
Epoch 7/10
4303/4303 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - accuracy: 0.0456 - loss: 12.6762 - val_accuracy: 0.0779 - val_loss: 12.4665
Epoch 8/10
4303/4303 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - accuracy: 0.0492

In [198]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [200]:
# Evaluate
print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Pred        BASEL  BELGRADE  BUDAPEST  MADRID
True                                         
BASEL        1474         2         4    2202
BELGRADE      955         0         0     137
BUDAPEST      196         0         1      17
DEBILT         82         0         0       0
DUSSELDORF     27         0         0       2
HEATHROW       72         0         0      10
KASSEL         11         0         0       0
LJUBLJANA      46         0         1      14
MAASTRICHT      5         0         0       4
MADRID        172         0         2     284
MUNCHENB        5         0         1       2
OSLO            4         0         0       1
STOCKHOLM       4         0         0       0
VALENTIA        0         0         0       1


## .12 CNN Model

In [203]:
epochs = 10
batch_size = 4
n_hidden = 4

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='softmax'))

C:\Users\jacks\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [207]:
model.summary()

Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_1 (Conv1D)               │ (None, 14, 4)          │            76 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 14, 16)         │            80 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 7, 16)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 112)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 15)             │         1,695 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,851 (7.23 KB)

 Trainable params: 1,851 (7.23 KB)

 Non-trainable params: 0 (0.00 B)

In [209]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [211]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/10
4303/4303 - 7s - 2ms/step - accuracy: 0.1178 - loss: 15360.5283
Epoch 2/10
4303/4303 - 5s - 1ms/step - accuracy: 0.1232 - loss: 133146.2812
Epoch 3/10
4303/4303 - 6s - 1ms/step - accuracy: 0.1341 - loss: 437385.2500
Epoch 4/10
4303/4303 - 6s - 1ms/step - accuracy: 0.1322 - loss: 980693.5625
Epoch 5/10
4303/4303 - 5s - 1ms/step - accuracy: 0.1277 - loss: 1830434.6250
Epoch 6/10
4303/4303 - 6s - 1ms/step - accuracy: 0.1264 - loss: 3043691.5000
Epoch 7/10
4303/4303 - 6s - 1ms/step - accuracy: 0.1221 - loss: 4702944.0000
Epoch 8/10
4303/4303 - 5s - 1ms/step - accuracy: 0.1222 - loss: 6862708.0000
Epoch 9/10
4303/4303 - 6s - 1ms/step - accuracy: 0.1251 - loss: 9523474.0000
Epoch 10/10
4303/4303 - 5s - 1ms/step - accuracy: 0.1197 - loss: 12737082.0000


In [213]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [215]:
# Evaluate
print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Pred        BELGRADE  BUDAPEST  DEBILT  KASSEL  LJUBLJANA  MAASTRICHT  MADRID  \
True                                                                            
BASEL            458      2027       9      43        117         326     383   
BELGRADE          10      1029       0       0          1          22      23   
BUDAPEST           2       202       0       0          0           4       5   
DEBILT             0        77       0       0          0           3       2   
DUSSELDORF         0        25       0       0          0           2       2   
HEATHROW           0        64       0       0          0           5      13   
KASSEL             0        10       0       0          0           1       0   
LJUBLJANA          0        54       0       0          1           2       4   
MAASTRICHT         0         8       0       0          1           0       0   
MADRID             5       259       1       1         11          4

#### There is already significant improvement with the number of stations identified using the CNN model compared to the RNN model

## .13 CNN Model Retest

In [219]:
epochs = 15
batch_size = 8
n_hidden = 8

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='softmax'))

C:\Users\jacks\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [221]:
model.summary()

Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_2 (Conv1D)               │ (None, 14, 8)          │           152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 14, 16)         │           144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 7, 16)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 112)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 15)             │         1,695 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,991 (7.78 KB)

 Trainable params: 1,991 (7.78 KB)

 Non-trainable params: 0 (0.00 B)

In [223]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [225]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/15
2152/2152 - 4s - 2ms/step - accuracy: 0.1039 - loss: 4673.3916
Epoch 2/15
2152/2152 - 3s - 1ms/step - accuracy: 0.1238 - loss: 45369.1133
Epoch 3/15
2152/2152 - 3s - 1ms/step - accuracy: 0.1338 - loss: 145794.1719
Epoch 4/15
2152/2152 - 3s - 1ms/step - accuracy: 0.1339 - loss: 323349.0000
Epoch 5/15
2152/2152 - 3s - 1ms/step - accuracy: 0.1410 - loss: 586685.8125
Epoch 6/15
2152/2152 - 3s - 1ms/step - accuracy: 0.1403 - loss: 950772.0000
Epoch 7/15
2152/2152 - 3s - 1ms/step - accuracy: 0.1391 - loss: 1427823.2500
Epoch 8/15
2152/2152 - 4s - 2ms/step - accuracy: 0.1392 - loss: 2041292.2500
Epoch 9/15
2152/2152 - 4s - 2ms/step - accuracy: 0.1368 - loss: 2796615.5000
Epoch 10/15
2152/2152 - 3s - 1ms/step - accuracy: 0.1366 - loss: 3736230.2500
Epoch 11/15
2152/2152 - 3s - 1ms/step - accuracy: 0.1396 - loss: 4842325.5000
Epoch 12/15
2152/2152 - 3s - 1ms/step - accuracy: 0.1389 - loss: 6121661.5000
Epoch 13/15
2152/2152 - 3s - 1ms/step - accuracy: 0.1397 - loss: 7623796.5000
Epoc

In [227]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [229]:
# Evaluate
print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Pred        BASEL  BELGRADE  BUDAPEST  DEBILT  DUSSELDORF  HEATHROW  KASSEL  \
True                                                                          
BASEL          21        15        28     126         469        35      83   
BELGRADE        1         9        10      20          67         0       0   
BUDAPEST        1         1         1       3          12         0       0   
DEBILT          0         0         0       2          10         0       0   
DUSSELDORF      1         0         0       1           8         0       0   
HEATHROW        0         0         0       2           8         0       0   
KASSEL          0         0         0       1           1         0       0   
LJUBLJANA       0         0         0       0           1         0       0   
MAASTRICHT      1         0         0       0           0         0       0   
MADRID          1         0         0       7          22         0       0   
MUNCHENB   

## .14 CNN Final Test

In [831]:
epochs = 15
batch_size = 8
n_hidden = 8

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = len(y_train[0])

model = Sequential()
model.add(Conv1D(n_hidden, kernel_size=2, activation='relu', input_shape=(timesteps, input_dim)))
model.add(Dense(16, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='tanh'))

C:\Users\jacks\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [833]:
model.summary()

Model: "sequential_62"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_52 (Conv1D)              │ (None, 14, 8)          │           152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_113 (Dense)               │ (None, 14, 16)         │           144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_52 (MaxPooling1D) │ (None, 7, 16)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_51 (Flatten)            │ (None, 112)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_114 (Dense)               │ (None, 15)             │         1,695 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,991 (7.78 KB)

 Trainable params: 1,991 (7.78 KB)

 Non-trainable params: 0 (0.00 B)

In [835]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [837]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/15
2152/2152 - 4s - 2ms/step - accuracy: 0.2782 - loss: 22.1699
Epoch 2/15
2152/2152 - 3s - 1ms/step - accuracy: 0.3105 - loss: 22.0853
Epoch 3/15
2152/2152 - 3s - 1ms/step - accuracy: 0.3122 - loss: 22.1395
Epoch 4/15
2152/2152 - 3s - 1ms/step - accuracy: 0.3140 - loss: 23.3270
Epoch 5/15
2152/2152 - 3s - 1ms/step - accuracy: 0.3224 - loss: 24.8392
Epoch 6/15
2152/2152 - 3s - 1ms/step - accuracy: 0.3074 - loss: 24.8186
Epoch 7/15
2152/2152 - 3s - 1ms/step - accuracy: 0.3121 - loss: 24.8195
Epoch 8/15
2152/2152 - 3s - 1ms/step - accuracy: 0.3145 - loss: 24.8186
Epoch 9/15
2152/2152 - 3s - 1ms/step - accuracy: 0.3166 - loss: 24.8176
Epoch 10/15
2152/2152 - 3s - 1ms/step - accuracy: 0.3204 - loss: 24.8186
Epoch 11/15
2152/2152 - 3s - 1ms/step - accuracy: 0.3229 - loss: 24.8176
Epoch 12/15
2152/2152 - 3s - 1ms/step - accuracy: 0.3308 - loss: 24.8186
Epoch 13/15
2152/2152 - 3s - 1ms/step - accuracy: 0.3366 - loss: 24.8186
Epoch 14/15
2152/2152 - 3s - 1ms/step - accuracy: 0.3412 - l

In [839]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [841]:
# Evaluate
print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Pred        BASEL  BUDAPEST  DUSSELDORF  KASSEL  MAASTRICHT  MADRID  MUNCHENB  \
True                                                                            
BASEL        1977        11         917     365           4       1         5   
BELGRADE     1084         0           8       0           0       0         0   
BUDAPEST      214         0           0       0           0       0         0   
DEBILT         82         0           0       0           0       0         0   
DUSSELDORF     29         0           0       0           0       0         0   
HEATHROW       81         0           1       0           0       0         0   
KASSEL         11         0           0       0           0       0         0   
LJUBLJANA      60         0           1       0           0       0         0   
MAASTRICHT      9         0           0       0           0       0         0   
MADRID        365         1          86       2           0       0 